In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
print("complete")

/kaggle/input/cvalltogether/CV_brief.pdf
/kaggle/input/cvalltogether/drmperfect_cv-april-2020.pdf
/kaggle/input/cvalltogether/vita_external.pdf
/kaggle/input/cvalltogether/Klar CV.pdf
/kaggle/input/cvalltogether/blee_cv_2016.pdf
/kaggle/input/cvalltogether/Canales_Robert_CV.pdf
/kaggle/input/cvalltogether/MOORE-MONROY2015_0.pdf
/kaggle/input/cvalltogether/CURRICULUM-VITAE_DHG_012519.pdf
/kaggle/input/cvalltogether/RobertsonCV0818-2.pdf
/kaggle/input/cvalltogether/hameroff2016cv_0.pdf
/kaggle/input/cvalltogether/Alison-M-Meadow-cv.pdf
/kaggle/input/cvalltogether/agaspar_cv.pdf
/kaggle/input/cvalltogether/JO - 2171.pdf
/kaggle/input/cvalltogether/Hoit CV (4-11-16).pdf
/kaggle/input/cvalltogether/Liverman Selected CV May 2018.pdf
/kaggle/input/cvalltogether/LBarraza CV 2020.pdf
/kaggle/input/cvpdfdata/agaspar_cv.pdf
complete


In [2]:
#all pip install
!pip install pdfplumber
print("complete")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 37.9 MB/s eta 0:00:00
complete


In [3]:
#all import
import pdfplumber
import pandas as pd
import os
import spacy
from spacy.matcher import Matcher

from datetime import datetime

import re
from nltk.corpus import stopwords

print("complete")

complete


In [4]:
#Name
#import spacy
#from spacy.matcher import Matcher

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)
def extract_name(resume_text):
    nlp_text = nlp(resume_text)
    
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', [pattern], on_match = None)
    
    matches = matcher(nlp_text)
    #res=[]
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        temp=span.text
        if ("vitae" in temp.lower() or "sciences" in temp.lower() 
            or "engineering" in temp.lower() or "biographical" in temp.lower() ):
            continue
        return temp
        #res.append(span.text)
    #return res
    
    

print("complete")

complete


In [5]:
#E-MAIL
#import re
def get_email_addresses(string):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return r.findall(string)


print("complete")

complete


In [6]:
#import re
#from datetime import datetime
def getDate(dataMaybeDate):
    

    match = re.match(r'.*([1-3][0-9]{3})', dataMaybeDate)
    if match is not None:
        # Then it found a match!
        #print(match.group(1))
        return match.group()
    return None

print("complete")

complete


In [7]:
#for Educational Institute
#import re

def findInstitute(tika_text):
    sub_patterns = ['[A-Z][a-z]* University',
                    '[A-Z][a-z]* Educational Institute',
                '[A-Z][a-z]* College',
                'University of [A-Z][a-z]*',
                'Ecole [A-Z][a-z]*',
                   '[A-Z][a-z]*University',
                    '[A-Z][a-z]*EducationalInstitute',
                '[A-Z][a-z]*College',
                'Universityof[A-Z][a-z]*',
                'Ecole[A-Z][a-z]*']
    pattern = '({})'.format('|'.join(sub_patterns))
    matches = re.findall(pattern, tika_text)

    return matches

print("complete")

complete


In [8]:
#import re
#from nltk.corpus import stopwords
# load pre-trained model
#nlp = spacy.load('en_core_web_sm')
# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))
# Education Degrees
EDUCATIONDEGREE = [
            'BE','B.E.', 'B.E', 'BS', 'B.S','B.S.','C.A.','B.COM','BCOM',
            'M.COM', 'MCOM','M.COM.',
            'ME', 'M.E', 'M.E.', 'MS', 'M.S','M.S.','MSC','M.SC','M.SC.',
            'BTECH', 'B.TECH','B.TECH.', 'M.TECH','M.TECH.', 'MTECH',
            'PHD','PH.D', 'PH.D.','MBA','GRADUATE', 
            'POST-GRADUATE','MASTERS',
            'SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII'
        ]
def extract_educationDegree(resume_text):
    nlp_text = nlp(resume_text)
    # Sentence Tokenizer
    nlp_text = [sent.text.strip() for sent in nlp_text.sents]
    edu = []
    
    # Extract education degree
    for index, text in enumerate(nlp_text):
        
        for tex in text.split():
            # Replace all special symbols
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            
            if tex.upper() in EDUCATIONDEGREE and tex not in STOPWORDS:
                
                if tex not in edu:
                    edu.append(tex)
                                
                
    return edu
print("complete")

complete


In [9]:
#import pdfplumber
#import pandas as pd
#import os

def extract_pdf(pdf_path):
    linesOfFile = []
    with pdfplumber.open(pdf_path) as pdf:
        for pdf_page in pdf.pages:
            single_page_text = pdf_page.extract_text()
            for line in single_page_text.split('\n'):
                linesOfFile.append(line)
                #print(linesOfFile)
    return linesOfFile


folder_with_pdfs = '../input/cvalltogether'
linesOfFiles = []



listOfPdfFiles=[]

for pdf_file in os.listdir(folder_with_pdfs):
    if pdf_file.endswith('.pdf'):
        #print(pdf_file)
        listOfPdfFiles.append(pdf_file)
        pdf_file_path = os.path.join(folder_with_pdfs, pdf_file)
        linesOfFile = extract_pdf(pdf_file_path)
        linesOfFiles.append(linesOfFile)
        
        
        
        
df = pd.DataFrame(linesOfFiles)
df.to_csv('test.csv')
print("complete")

complete


In [10]:
listOfPdfFiles

['CV_brief.pdf',
 'drmperfect_cv-april-2020.pdf',
 'vita_external.pdf',
 'Klar CV.pdf',
 'blee_cv_2016.pdf',
 'Canales_Robert_CV.pdf',
 'MOORE-MONROY2015_0.pdf',
 'CURRICULUM-VITAE_DHG_012519.pdf',
 'RobertsonCV0818-2.pdf',
 'hameroff2016cv_0.pdf',
 'Alison-M-Meadow-cv.pdf',
 'agaspar_cv.pdf',
 'JO - 2171.pdf',
 'Hoit CV (4-11-16).pdf',
 'Liverman Selected CV May 2018.pdf',
 'LBarraza CV 2020.pdf']

In [11]:
df

,0,1,2,3,4,5,6,7,8,9,...,3575,3576,3577,3578,3579,3580,3581,3582,3583,3584
0,Sunder Sethuraman,Brief Vitæ,E-Mail: sethuram@math.arizona.edu,WWW: http://www.math.arizona.edu/~ sethuram/,Education,"1995 Ph.D. Courant Institute, New York Univers...","1990 B.S. Stanford University, Stanford, CA.","1986 Diploma Florida High School, Tallahassee,...",Professional Experience,"2011 – Professor Department of Mathematics,",...,None,None,None,None,None,None,None,None,None,None
1,,"Michelle M. Perfect, Ph.D.",,University Address: 1430 E. 2nd St Cell Phone...,University of Arizona Email: mperfect@email.a...,"Tucson, AZ 85721",,Chronology of Education,2005 – 2008 Texas A&M continuing education in...,"certificate program (neuroanatomy, immunology,",...,None,None,None,None,None,None,None,None,None,None
2,Curriculum Vitae,Thomas Garrett Kennedy,Department of Mathematics,University of Arizona,"Tucson, Arizona 85721, USA",Phone: 520-626-0197,e-mail: tgk@math.arizona.edu,∼,Home page: www.math.arizona.edu/ tgk,Education,...,None,None,None,None,None,None,None,None,None,None
3,,Updated: November 2020,,,,Samara Klar,Phone: 262-372-6726,Email: klar@email.arizona.edu,Web: www.samaraklar.com,EMPLOYMENT,...,None,None,None,None,None,None,None,None,None,None
4,"Benjamin R. Lee, M.D.",,,CURRICULUM VITAE,,"Name Benjamin R. Lee, M.D.",,Present Position Professor and Chief of Urolo...,University of Arizona College of Medicine,,...,320. Video‐ rate structured illumination micr...,"core needle renal biopsy, Weil Lai, Mei Wang, ...","Benjamin Lee.. PD14-05, AUA national meeting,...",,321. Nanotechnology combination therapy for r...,inhibitor produces synergistic treatment respo...,"Weil Lai*, Caleb Abshire, Sree Harsha Mandava,...","Benjamin Lee. PD46-06. AUA national meeting,...",Page 76 of 76,
5,CURRICULUM VITAE,of,ROBERT A. CANALES,,Chronology of Education,"1996 BS, with Honors, Civil Engineering, The ...",,"1997 MS, Civil & Environmental Engineering, S...",,"2000 MS, Statistics, Stanford University",...,None,None,None,None,None,None,None,None,None,None
6,,OMB No. 0925-0001/0002 (Rev. 08/12 Approved Th...,BIOGRAPHICAL SKETCH,Provide the following information for the Seni...,Follow this format for each person. DO NOT EX...,,NAME POSITION TITLE,Martha J. Moore-Monroy Program Director for t...,"the University of Arizona, Center of Excellenc...","eRA COMMONS USER NAME (credential, e.g., agenc...",...,None,None,None,None,None,None,None,None,None,None
7,CURRICULUM VITAE Dawn H. Gouge Ph.D. B.Sc.,,Current Employer and Position,,Professor and Specialist,Public Health Pest Entomologist and Integrated...,"University of Arizona, Department of Entomology.",,"Physical Location MAC Experiment Station, 3786...",85138. (520) 374-6223; Fax (520) 374-6394; La...,...,None,None,None,None,None,None,None,None,None,None
8,CHRISTOPHER T. ROBERTSON,chris.robertson@law.arizona.edu,http://ssrn.com/author=769387,520-621-1289 (office),@ProfCRobertson (Twitter),I. CURRENT POSITIONS AND AFFILIATIONS,"Professor of Law, Associate Dean for Research ...",Regulatory Science Program; James E. Rogers Co...,(2010-),,...,None,None,None,None,None,None,None,None,None,None
9,Biographical Sketch – Full List of Publication...,,Name,,"Stuart R. Hameroff, MD",,Position title,,"Emeritus Professor, Anesthesiology","Emeritus Professor, Psychology",...,None,None,None,None,None,None,None,None,None,None


In [12]:
# to do : do partial match , i.e. word+etc

reference=["advising","articles","awards","appointments","affiliations",
           "books","chapters","contacts","commentaries",
           "current","conferences","citations","certificates",
           "certifications","editorials","committees","educations",
           "experiences","employments","editorials",
           "grants","honors","invited","journals","licenses","meetings",
           "memberships","newsletters","outreaches","observing",
           "presentations","publications","proposals","postdocs",
           "postdoctorals","practices","professionals","researches",
           "reports","services","students","scholarships",
           "sponsored","supervision","softwares","seminars","scholarly",
           "teaching","talks","works"]
#to store data
allCVData=[]
allCVData.append(("fileName","sectionName","personName","head","relation/lebel","tail"))


print("complete")

complete


In [13]:
def dfPartByPart(textList):
    collectedText=[]
    collectedByDictionary=dict()
    collectedParts=dict()
    keyDictionary="introductions"
    temp=[]



    for smallPartText in textList:
        if smallPartText==None:
            continue
   
        for partOfSmallPartText in smallPartText.split():
            if len(partOfSmallPartText)<2:
                continue
            result = list(filter(lambda x: x.startswith(partOfSmallPartText.lower()), reference))
            if result:
                
                collectedText.append(temp)
                if keyDictionary in collectedByDictionary:
                    collectedByDictionary[keyDictionary]=collectedByDictionary[keyDictionary]+temp
                else:
                    collectedByDictionary[keyDictionary]=temp
                keyDictionary=result[0]
                temp=[]
                break
        
        temp.append(smallPartText)
    
    
    collectedText.append(temp)
    if keyDictionary in collectedByDictionary:
        collectedByDictionary[keyDictionary]=collectedByDictionary[keyDictionary]+temp
    else:
        collectedByDictionary[keyDictionary]=temp

    return collectedByDictionary
print("complete")

complete


In [14]:
def getDataTriple(fileName,sectionName,personName,collectedByDictionary,relation):
    tempData=collectedByDictionary[relation]
    
    for partData in tempData:
        tempText=partData.split("\n")
        
        for smallerPartData in tempText:
            
            tempTriple=(fileName,sectionName,personName,relation,smallerPartData)
            if tempTriple not in allCVData:
                allCVData.append(tempTriple)
            
            # try to get more data
            
            tempCVData=smallerPartData.split(",")
            
            xtemp=getDate(smallerPartData)
            if xtemp:

                smallerPartData=smallerPartData.split(",")
                if len(smallerPartData)<1:
                    continue
                
                
                #temp002=(relation,"done in",xtemp)
                temp002=(fileName,sectionName,relation,"dated in",xtemp)
                
                allCVData.append(temp002)
                if len(smallerPartData[0])>0:
                    temp003=(fileName,sectionName,personName,"as",smallerPartData[0])
                    allCVData.append(temp003)
                
                
                for lastPartData in smallerPartData:
                    if len(lastPartData)>0:
                        temp004=(fileName,sectionName,relation,"with",lastPartData)
                        allCVData.append(temp004)
            


print("complete")

complete


In [15]:
def handleEducations(pdfFileName,CVPersonName,sectionType,sentParts):
    #print(sentParts)
    for parts in sentParts:
        if parts==None or len(parts)<2:
            continue
        allCVData.append((pdfFileName,CVPersonName,sectionType,"educations","in",parts))
        dates=getDate(parts)
        educationDegree=extract_educationDegree(parts)
        institutes=findInstitute(parts)
        if len(educationDegree)>0:
            for degree in educationDegree:
                if len(institutes)>0:
                    for university in institutes:
                        allCVData.append((pdfFileName,CVPersonName,sectionType,degree,"with",university))
                if len(dates)>0:
                    for newDate in dates:
                        allCVData.append((pdfFileName,CVPersonName,sectionType,degree,"in",newDate))
print("complete")


complete


In [16]:
def getTextData(sentList):
    res=[]
    for i in sentList:
        if i!=None:
            res.append(i)
    return ",".join(res)

In [17]:
matrix_res = df.to_numpy()
#matrix_res = df.to_records()
#print(matrix_res[5])
#print(matrix_res[11][4])
#print(listOfPdfFiles)
print(len(listOfPdfFiles))

print(len(matrix_res))

for i in range(len(matrix_res)):
    pdfFileName=listOfPdfFiles[i]
    
    sentData=getTextData(matrix_res[i])
    CVPersonName=extract_name(sentData)
    
    sectionDictionary=dfPartByPart(matrix_res[i])
    
    personEMail=get_email_addresses(sentData)
    CVPersonName=extract_name(sentData)
    
    for sectionType in sectionDictionary:
        getDataTriple(pdfFileName,CVPersonName,sectionType,sectionDictionary,sectionType)
        if sectionType=="educations":
            handleEducations(pdfFileName,CVPersonName,sectionType,sentData)
    


#print(matrix_res)

16
16


In [18]:
allCVData[:5]

[('fileName', 'sectionName', 'personName', 'head', 'relation/lebel', 'tail'),
 ('CV_brief.pdf',
  'Sunder Sethuraman',
  'introductions',
  'introductions',
  'Sunder Sethuraman'),
 ('CV_brief.pdf',
  'Sunder Sethuraman',
  'introductions',
  'introductions',
  'Brief Vitæ'),
 ('CV_brief.pdf',
  'Sunder Sethuraman',
  'introductions',
  'introductions',
  'E-Mail: sethuram@math.arizona.edu'),
 ('CV_brief.pdf',
  'Sunder Sethuraman',
  'introductions',
  'introductions',
  'WWW: http://www.math.arizona.edu/~ sethuram/')]

In [19]:
import csv

#write.csv(allCVData, "mycsv.csv")

#header = ['head','relation','tail']

with open('CVpdfDataVersion002.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header , already has a header
    #writer.writerow(header)

    # write multiple rows
    writer.writerows(allCVData)

path001 = r"./CVpdfDataVersion002.csv"
#using Tika
#pip install tika
#from tika import parser
#raw001 = parser.from_file(path001)
#print(raw001['content'])

print("complete")

complete
